In [1]:
"""
1. 모델 클래스 선택
2. 모델 초모수 선택
3. 모델을 훈련 데이터에 적합
4. 모델을 사용해서 새 데이터에 대한 레이블 예측
""";

# 처음 두 단계인 모델 선택과 초모수 선택이 도구와 기법을 효과적으로 사용하는 데 가장 중요한 부분

In [2]:
# 모델 검증에 대한 고려사항

# 잘못된 방식의 모델 검증
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

# k-nearest neighbor (n_neighbors = 1)
from sklearn.neighbors import KNeighborsClassifier as KNN
model = KNN(n_neighbors=1)
model.fit(X, y)
y_model = model.predict(X)

In [3]:
from sklearn.metrics import accuracy_score
accuracy_score(y_model, y)

# 100% => "같은 데이터로 모델을 훈련하고 검증한다" 라는 근본적인 오류
# 100% => nearest neighbor model은 단순히 훈련 데이터를 저장하고 점들과 새로운 데이터 비교, 인스턴스 기반 추정

1.0

In [4]:
# 올바른 방식의 모델 검증: 검정 표본
# 검정 표본(holdout set)

from sklearn.model_selection import train_test_split
# 데이터를 각각 50%로 나눔
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, train_size=0.5)

# 모델을 이 가운데 하나의 데이터 집합에 적합
model.fit(X_train, y_train)

# 모델을 두 번째 데이터 집합으로 검증함
y2_model = model.predict(X_test)

In [5]:
accuracy_score(y2_model, y_test)
# 90% 정확도 => 검정 표본은 알려지지 않은 데이터와 비슷

0.9066666666666666

In [6]:
# 교차 검증을 통한 모델 검증
# cross-validation => 데이터의 각 하위 집합이 훈련 자료와 검증 자료로 사용 되도록 일련의 적합

X1, X2, Y1, Y2 = train_test_split(X, y, random_state=0, train_size=0.5) # 2겹 교차 검증

y2_model = model.fit(X1, Y1).predict(X2)
y1_model = model.fit(X2, Y2).predict(X1)

accuracy_score(y2_model, Y2), accuracy_score(y1_model, Y1)

(0.9066666666666666, 0.96)

In [7]:
# SciKit-Learn의 cross_val_score 루틴
from sklearn.model_selection import cross_val_score
cross_val_score(model, X, y, cv=5) # 5겹 교차 검증

array([0.96666667, 0.96666667, 0.93333333, 0.93333333, 1.        ])

In [8]:
# 극단적인 교차 검증 => 데이터 점의 개수와 같은 수만큼 반복
# 단일 관측치 제거 방식 (leave-one-out)
from sklearn.model_selection import LeaveOneOut
scores = cross_val_score(model, X, y, cv=LeaveOneOut)
scores.mean() # ~96% 정확도

AttributeError: 'numpy.ndarray' object has no attribute '_iter_test_masks'

In [ ]:
# 최적의 모델 선택하기
"""
추정기의 성과가 저조하다면 어떻게 개선할 것인가?
=> 더 복잡학허나 더 유연한 모델 사용
=> 덜 복잡하거나 덜 유연한 모델 사용
=> 더 많은 훈련 표본 수집
=> 각 표본에 특징을 추가하기 위해 더 많은 데이터 수집
""";

# 편향-분산 트레이드오프
# 편한과 분산 사이의 트레이드오프에서 가장 효율적인 점 찾기

"""
과소적합(underfit)
=> 직선보다 더 복잡한 데이터를 직선모델로 설명 하기
=> 데이터의 특징을 적절히 설명할 만큼 모델 유연성이 충분하지 않다

과적합(overfit)
=> 모델이 기본 데이터 분포와 함께 임의의 오류까지 설명할 정도로 너무 과한 모델 유연성을 가지고 있다
=> 이 모델은 고분산을 가지고 있다

표본 점수 (R-squared value)
=> 모델이 대상값의 단순 평균에 비해 얼마나 잘 수행하는지 측정
=> 1(완전히 일치함)
=> 0(모델이 데이터의 단순 평균을 구하는 수준)
=> 음수 (나쁜 모델)
""";

# 검증 곡선 (validation curve)
"""
1. 훈련 점수는 언제나 검증 점수보다 높다 (모델은 이미 본 데이터에서 더 잘 적합)
2. 모델 복잡도가 너무 낮은 경우(고편향), 데이터가 과소적합(underfit) => 훈련 / 테스트 데이터 둘다 예측 못 함
3. 모델 복잡도가 너무 높은 경우(고분산), 데이터가 과적합(overfit) => 훈련 데이터 예측 성공 / 테스트 데이터 실패
""";

In [ ]:
# SciKit-Learn의 검증 곡선
# 다항 회귀(polynomial regression) 모델 사용
"""
1차 => y = ax + b
2차 => y = ax^2 + bx + c
3차 => y = ax^3 + bx^2 + cx + d
...

SciKit-Learn에서는 이것을 다항식 전처리 프로그램과 결합한 간단한 선형 회구로 구현
연산을 하나로 묶는 파이프라인을 사용
""";

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), LinearRegression(**kwargs))

In [ ]:
import numpy as np

def make_data(N, err=1.0, rseed=1):
    # 임의로 데이터 표본 만들기
    rng = np.random.RandomState(rseed)
    X = rng.rand(N, 1) ** 2
    y = 10-1./(X.ravel()+0.1)
    if err > 0:
        y += err * rng.randn(N)
    return X, y

X, y = make_data(40)

In [ ]:
# 데이터와 함께 여러 차수의 다항식 적합을 시각화
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set() # 플롯 형식 지정

X_test = np.linspace(-0.1, 1.1, 500)[:, np.newaxis]

plt.scatter(X.ravel(), y, color="black")
axis = plt.axis()
for degree in [1, 3, 5]:
    y_test = PolynomialRegression(degree).fit(X, y).predict(X_test)
    plt.plot(X_test.ravel(), y_test, label="degree{0}".format(degree))
plt.xlim(-0.1, 1.0)
plt.ylim(-2, 12)
plt.legend(loc="best");

In [ ]:
# validation_curve
from sklearn.model_selection import validation_curve
degree = np.arange(0, 21)
train_score, val_score = validation_curve(PolynomialRegression(), X, y,
                                          param_name="polynomialfeatures_degree", param_range=degree, cv=7)
plt.plot(degree, np.median(train_score, 1), color="blue", label="training score")
plt.plot(degree, np.median(val_score, 1), color="red", label="validation score")
plt.legend(loc="best")
plt.ylim(0, 1)
plt.xlabel("degree")
plt.ylabel("score");

In [ ]:
# 검증 곡선 => 3차 다항식이 최적의 트레이드오프
plt.scatter(X.ravel(), y)
lim = plt.axis()
y_test = PolynomialRegression(3).fit(X, y).predict(X_test)
plt.plot(X_test.ravel(), y_test);
plt.axis(lim);

In [ ]:
# 학습 곡선
X2, y2 = make_data(200)
plt.scatter(X2.ravel(), y2);

In [ ]:
# validation_curve
degree = np.arange(0, 21)
train_score2, val_score2 = validation_curve(PolynomialRegression(), X2, y2,
                                          param_name="polynomialfeatures_degree", param_range=degree, cv=7)
plt.plot(degree, np.median(train_score2, 1), color="blue", label="training score")
plt.plot(degree, np.median(val_score2, 1), color="red", label="validation score")
plt.plot(degree, np.median(train_score, 1), color="blue", alpha=.3, linestyle="dashed")
plt.plot(degree, np.median(val_score, 1), color="red", alpha=.3, linestyle="dashed")
plt.legend(loc="lower center")
plt.ylim(0, 1)
plt.xlabel("degree")
plt.ylabel("score");

In [ ]:
# 더 큰 규모의 데이터세트가 훨씬 더 복잡한 모델을 지원할 수 있다
# 검증 곡선의 행동은 두개의 입력값에 의해 결정 => 1. 모델 복잡도 2. 훈련 데이터 점의 개수
# 훈련 집합의 크기에 따른 훈련 점수/검증 점수의 플롯을 학습 곡선(learning curve) 이라고 한다
"""
해당 복잡도의 모델은 작은 데이터세트를 과적합 => 훈련점수는 높지만 검증점수는 낮다
해당 복잡도의 모델은 큰 데이터세트를 과소적합 => 훈련점수는 감소하지만 검증점수는 증가한다

특정 모델이 수렴할 만큼 충분한 데이터 점을 가지게 되면 훈련 데이터를 더 늘리는 것은 도움 X"
=> 더 복잡한 모델을 사용할 것
""";

In [ ]:
# SciKit-Learn의 학습 곡선
from sklearn.model_selection import learning_curve

fig, ax = plt.subplots(1, 2, figsize=(16,6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

for i, degree in enumerate([2, 9]):
    N, train_lc, val_lc = learning_curve(PolynomialRegression(degree), X, y, cv=7,
                                         train_sizes=np.linspace(0.3, 1, 25))
    ax[i].plot(N, np.mean(train_lc, 1), color="blue", label="training score")
    ax[i].plot(N, np.mean(val_lc, 1), color="red", label="validation score")
    ax[i].hlines(np.mean([train_lc[-1], val_lc[-1]]), N[0], N[-1], color="gray",
                 linestyle="dashed")

    ax[i].set_ylim(0, 1)
    ax[i].set_xlim(N[0], N[-1])
    ax[i].set_xlabel("training size")
    ax[i].set_ylabel("score")
    ax[i].set_title("degree = {0}".format(degree), size=14)
    ax[i].legend(loc="best")

In [ ]:
# 훈련 데이터를 더 많이 추가한다고 해서 적합도가 눈에 띄게 개선되지 않을 것이다
# 더 복잡한 모델로 옮겨가면 수렴 점수는 높아지지만 모델편차는 높아지는 것을 감수해야 한다.
# => 월씬 더 많은 데이터를 추가하면 더욱 복잡한 모델의 학습 곡선도 결국에는 수렴한다.

In [ ]:
# 실제 검증: 그리드 검색
# grid_search 모듈

# 자동 그리드 검색을 통해 결정된최적합 모델

from sklearn.model_selection import GridSearchCV

param_grid = {"polynomialfeatures_degree": np.arange(21),
              "linearregression_fit_intercept": [True, False],
              "linearregression_normalize": [True, False]}

grid = GridSearchCV(PolynomialRegression(), param_grid, cv=7)

In [ ]:
grid.fit(X, y);

grid.best_params_

In [ ]:
model = grid.best_estimator_

plt.scatter(X.ravel(), y)
lim = plt.axis()
y_text = model.fit(X, y).predict(X_test)
plt.plot(X_test.ravel(), y_test, hold=True);
plt.axis(lim);